<a href="https://colab.research.google.com/github/alexdoss/git_practivce/blob/main/Comparaison_r%C3%A9glage_vente_pot_arrondie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math

#N'oublie pas de nommer la première colonne
#remplacer les espace par un tiret du six manuellement dans le csv et charger le csv dans collab puis appliquer le code ci-dessous
forecast = pd.read_csv("/content/fichier_test.csv", delimiter=";")
def eliminate_nan(forecast):
  return forecast.dropna()

forecast1 = eliminate_nan(forecast)

data_analyzed = forecast1[['ID', 'Nitro', '05/09/2024', '05/09/2024_fournis', '05/09/2024_rupture']].copy()


#cette fonction calcul les invendus potentiel. hypothèse vente potentielle = 1.2
#Pour utiliser cette fonction, définissez la date de référence de la comparaison Nitro vs réel et débrouille toi pour la remplacer par la date dont tu as besoin. Concrètement, change le nom de la colonne.
def function_invendu_pot(row):
  if ((row['05/09/2024_rupture'] == True) & ((row['Nitro']-math.ceil(row['05/09/2024']*1.2))<=0)):
    return 0
  elif ((row['05/09/2024_rupture'] == True) & ((row['Nitro']- math.ceil(row['05/09/2024']*1.2))>0)):
    return row['Nitro']-math.ceil(row['05/09/2024']*1.2)
  elif ((row['05/09/2024_rupture']== False) & (row['Nitro']<row['05/09/2024'])):
    return 0
  else:
    return row['Nitro']-row['05/09/2024']


data_analyzed['invendu_potentiel']=data_analyzed.apply(function_invendu_pot, axis=1)
data_analyzed['rupture_potentielle']=data_analyzed['invendu_potentiel'].apply(lambda x: True if x == 0 else False)

def function_vente_potentielle(row):
  if ((row['05/09/2024_rupture'] == True) & ((row['Nitro']-math.ceil(row['05/09/2024']*1.2))<=0)):
    return row['Nitro']
  elif ((row['05/09/2024_rupture'] == True) & ((row['Nitro']-math.ceil(row['05/09/2024']*1.2))>0)):
    return math.ceil(row['05/09/2024']*1.2)
  elif ((row['05/09/2024_rupture']== False) & (row['Nitro']<row['05/09/2024'])):
    return row['Nitro']
  else :
    return row['05/09/2024']

data_analyzed['vente_potentielle']=data_analyzed.apply(function_vente_potentielle, axis=1)

invendu_potentiel= np.sum(data_analyzed['invendu_potentiel'])/np.sum(data_analyzed['Nitro'])
vente_potentielle = np.sum(data_analyzed['vente_potentielle'])
taux_rupture_potentielle = np.sum(data_analyzed['rupture_potentielle']==True)/len(data_analyzed['rupture_potentielle'])
VAZ_potentielle = data_analyzed.loc[data_analyzed['rupture_potentielle']==True,'vente_potentielle'].sum()/data_analyzed['vente_potentielle'].sum()

synthesis_table = [
    {
    'invendu_potentiel': invendu_potentiel,
    'vente_potentielle': vente_potentielle,
    'taux_rupture_potentielle': taux_rupture_potentielle,
    'VAZ_potentielle': VAZ_potentielle
}
]

synthesis = pd.DataFrame(synthesis_table)
print(synthesis)
synthesis.to_csv('synthesis.csv', index=False)
data_analyzed.to_csv('data_analyzed.csv', index=False)

   invendu_potentiel  vente_potentielle  taux_rupture_potentielle  \
0           0.347537            39091.0                   0.20171   

   VAZ_potentielle  
0         0.264997  


In [ ]:
tx_invendu_reel = (np.sum(data_analyzed['05/09/2024_fournis'])-np.sum(data_analyzed['05/09/2024']))/np.sum(data_analyzed['05/09/2024_fournis'])
vente_reel = np.sum(data_analyzed['05/09/2024'])
tx_rupture_reel = np.sum(data_analyzed['05/09/2024_rupture']==True)/len(data_analyzed['05/09/2024_rupture'])
VAZ_reel = data_analyzed.loc[data_analyzed['05/09/2024_rupture']==True,'05/09/2024'].sum()/data_analyzed['05/09/2024'].sum()

reel_synthesis_table = [
    {
    'tx_invendu_reel': tx_invendu_reel,
    'vente_reel': vente_reel,
    'tx_rupture_reel': tx_rupture_reel,
    'VAZ_reel': VAZ_reel
}
]
reel_synthesis = pd.DataFrame(reel_synthesis_table)
print(reel_synthesis)
reel_synthesis.to_csv('reel_synthesis_table.csv', index=False)

   tx_invendu_reel  vente_reel  tx_rupture_reel  VAZ_reel
0         0.270508       35314         0.150546  0.165827
